<h1 style="font-family: Arial, Helvetica, sans-serif; text-align:center;">Lightning Dashboard <h1>

<hr style="background-color: black; height: 2px; border: none; margin: 5px 0; font-weight: bold;">
<h1 style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    **Showing how COG (Cloud Optimized Geotiff Images) are visualized with mapbox using TRMM_LIS_FULL as a sample example.**
</h1>
<hr style="background-color: black; height: 2px; border: none; margin: 5px 0; font-weight: bold;">

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Optionally installing the required libraries depending on whether the system already has it
</span>

In [1]:
import importlib

# List of packages to install
packages_to_install = [
    'plotly',
    'rasterio',
    'pandas',
    'requests',
    'xarray',
    'earthaccess',
    'gzip'
]

for package_name in packages_to_install:
    if importlib.util.find_spec(package_name) is None:
        !pip install {package_name}
    else:
        print(f"{package_name} is already installed.")

plotly is already installed.
rasterio is already installed.
pandas is already installed.
requests is already installed.
xarray is already installed.
earthaccess is already installed.
gzip is already installed.


<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold">
    Importing library used for downloading TRMM LIS FULL dataset from EDS.
</span>

In [2]:
# Suppress warnings
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

# Direct access
import earthaccess as ea
import os
import random
import gzip

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold">
    Functions below downloads the TRMM LIS FULL dataset directly from EDS and then unzips it.
</span>

In [3]:
#pull trmm_lis_full from EDS directly
def download_data():
    auth = ea.login(persist=True) # use 'interactive' strategy the first time to log in
    DAAC2 = 'GHRC_DAAC'
    #doi of dataset
    doi = "10.5067/LIS/LIS/DATA301"
    
    # step2: get links #
    results2 = ea.search_data(
        doi=doi,
        cloud_hosted=True,
    )
    
    # data_link list
    https_links2 = []  # external link, todo: find ways to directly access data using external link

    for granule in results2:
        https_links2.extend(granule.data_links(access="on_prem"))

    https_links2 = [file for file in https_links2 if file.endswith(".nc.gz")]#only select .nc files
    file_count = len(https_links2)
    folder_name = f"./local_folder{random.randint(1, 10000)}" 
    print(os.getcwd())
    if file_count != 0:
        os.mkdir(folder_name)#define folder to store data
        ea.download(https_links2, folder_name)#download_data
        print(f'Data downloaded on folder: {folder_name}')
        
    #unzip the .gz files
    filename = unzip(os.path.join(os.getcwd(), folder_name))
    
    return file_count, folder_name, filename

#unzipping the files in a directory
def unzip(path):
    original_cwd = os.getcwd()
    os.chdir(path)
    files = os.listdir()
    
    for file in files:
        if file.endswith('.gz'):
            with gzip.open(file, 'rb') as f_in:
                with open(file[:-3], 'wb') as f_out:
                    f_out.writelines(f_in)
                    
    for file in files:
        if file.endswith('.gz'):
            os.remove(file)
    files = os.listdir()
    os.chdir(original_cwd)        
    return files

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold">
    Calling the download_data function. Data gets downloaded and saved inside a local folder. Might ask for authentication. 
</span>

In [4]:
file_count, folder_name, filename = download_data()

EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
Unable to parse .netrc
Enter your Earthdata Login username: alansubedi
Enter your Earthdata password: ········
You're now authenticated with NASA Earthdata Login
Using token with expiration date: 12/19/2023
Using user provided credentials for EDL
Persisting credentials to .netrc
Granules found: 1
/home/asubedi/Desktop/work/GHRCCLOUD-5331_JupyterNotebook_for_LD


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Data downloaded on folder: ./local_folder3497


<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold">
    Converting the TRMM LIS FULL netcdf4 file to COG. 
</span>

In [12]:
import xarray as xa

file_path = os.path.join(os.getcwd(), folder_name, filename[0])

def convert_to_COG(file_path, folder_name):
    file = xa.open_dataset(file_path, engine='netcdf4', decode_coords='all')
    var = "VHRFC_LIS_FRD"
    grid = file[var]
    grid = grid[::-1] # Orientation is flipped to the correct position
        
    grid.rio.set_spatial_dims(x_dim='Longitude', y_dim='Latitude', inplace=True)
    grid.rio.crs
    grid.rio.set_crs('epsg:4326')
    cog_name = f'{var}_co.tif'
    cog_path = os.path.join(os.getcwd(), folder_name, cog_name)
    grid.rio.to_raster(rf'{cog_path}', driver='COG')
    
#calling the conversion function
convert_to_COG(file_path, folder_name)

print(f"--> netcdf4 file and the COG are both stored at directory: \n   {os.path.join(os.getcwd(), folder_name)}")

ERROR 1: PROJ: internal_proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


--> netcdf4 file and the COG are both stored at directory: 
   /home/asubedi/Desktop/work/GHRCCLOUD-5331_JupyterNotebook_for_LD/./local_folder3497


<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold">
    COGs are then added to the s3 bucket and the tiling server (terracotta) is configured then deployed to the AWS. We now get started with the frontend.
</span>

In [6]:
#base url of the deployed terracotta server
base_url = "https://wug8w3fg42.execute-api.us-west-2.amazonaws.com/development/singleband/"

#url for the trmm lis full dataset
trmm_lis_full = "VHRFC/201301/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.325,23.426]"

#sample url for other datasets (proper date/time need to be added to each url to get proper COG response from the server)
trmm_lis_seasonal = "VHRSC/{date}/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.00010455249866936,0.06766455620527]"
otd_monthly = "HRMC/{date}/OTD/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,0.2]"
isslis = "ISS_LIS/{date}/{time}/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,0]"
nalma = "NALMA_{time}/{date}/{band}/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,1]"

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Importing Libraries used for Visualization
</span>

In [7]:
import plotly.io as pio
import plotly.express as px
import rasterio
from rasterio.plot import show
import pandas as pd
import requests

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Function Below visualizes COG on mapbox. COG image url needs to be passed
</span>

In [8]:
#this function visualized any given COG inside of a mapbox view
def visualize_COG(layer_url, zoom=1, center={"lat": 33.543682, "lon": -86.779633}):
    pio.renderers.default = 'iframe'
    styles_list = [
        "carto-darkmatter",
        "carto-positron",
        "open-street-map",
        "stamen-terrain",
        "stamen-toner",
        "stamen-watercolor",
        "white-bg"
    ]
    style = styles_list[1]
    # seed value, for mapbox to load in colab
    df = pd.DataFrame([[1001, 5.3],[1001, 5.3]])
    df.columns = ["flips", "unemp"]
    fig = px.choropleth_mapbox(
                                df, 
                                color='unemp',
                                color_continuous_scale="Viridis",
                                range_color=(0, 12),
                                mapbox_style=style,
                                center=center,
                                opacity=1,
                                zoom=zoom 
                              )
    fig.update_layout(
        mapbox_layers=[
            {
                "sourcetype": "raster",
                "source": [layer_url]
            }
          ])
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(mapbox_style=style)
    fig.layout.mapbox.zoom = zoom
    fig.show()

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Calling the visualize function (TRMM LIS FULL is visualized on the Mapbox)
</span>

In [9]:
visualize_COG(base_url+trmm_lis_full)

<hr style="background-color: black; height: 2px; border: none; margin: 5px 0; font-weight: bold;">
<h1 style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Package below uses the same concept as above to visualize all the current available datasets on the Lightning Dashboard.
</h1>
<hr style="background-color: black; height: 2px; border: none; margin: 5px 0; font-weight: bold;">

In [10]:
from LDVisualization.inputHandlers import drop_down

In [11]:
drop_down()

Select a dataset from the options below: 

Dropdown(options=('TRMM_LIS_FULL', 'TRMM_LIS_SEASONAL', 'TRMM_LIS_MONTHLY', 'TRMM_LIS_DIURNAL', 'TRMM_LIS_DAIL…

Output()